<a href="https://colab.research.google.com/github/beekal/MachieneLearningProjects/blob/master/0%20Basics%20-%20TF/TF_2_0_Basics%20Revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF 2.0 Topics Covered
1. TF.Data : A single point of entry to handle any/ varied data type ranging from pandas, csv, image, text, TfRecordByte e.t.c
2. TFX: Tensorflow extended,  which aims to  provided end to end TF ecosystem from its  research/ prototype to  the server deployment. It includes the following  all of  which we will cover in this notebook
  -  TF validation : Used to validate the data
  -  TF Transform : Used to transform the data to its modeling state
  -  TF Modeling / Analysis : Used to create the model and evaluate its performance. Reiterate until satisfactory Metric reached.
  - TF Serving: Used to serve the model to production with versioning/ Rollback capabilities.

REF: https://www.tensorflow.org/tfx 

In [2]:
import tensorflow as tf
import numpy

print(f'TF version : ', tf.__version__)

TF version :  2.2.0-rc3


## TF.Data
Depending on the data source you wil have to use different TF data load calls
  - Data in memory :
      - tf.data.Dataset.from_tensors() or
      - tf.data.Dataset.from_tensor_slices()

In [44]:
d_mem = tf.data.Dataset.from_tensor_slices([1,2,3])
print(f' Tf.Data from memory : ' ,[e.numpy() for e in d_mem])

it = iter(d_mem)
print(f' Tf.Data from memory using iterator : ' ,next(it).numpy())

 Tf.Data from memory :  [1, 2, 3]
 Tf.Data from memory using iterator :  1


### TF.Data: From Mixed datatype
  - Use generator to convert each elements into a tf.Data Dataset 

In [45]:
person_data=  ([{ 'age':18,'name':'Hary' },
              { 'age':30,'name':'Sam' }
              ])
person_data
person =  tf.data.Dataset.from_generator( lambda: person_data, {"age": tf.int32, "name":tf.string}  )
print(person)
print(list(person.as_numpy_iterator()))


<FlatMapDataset shapes: {age: <unknown>, name: <unknown>}, types: {age: tf.int32, name: tf.string}>
[{'age': 18, 'name': b'Hary'}, {'age': 30, 'name': b'Sam'}]


## TF Transform :
We can apply different type of transformations as per our need
  - Dataset.map() : Per-Element operations
  - Dataset.filter(): Per-element Filter Operation
  - Dataset.reduce(): Reduce transfomations to single scalar value
  - Dataset.batch(): Per-batch operations

### Dataset.map: Parallelisation/ SpeedUp
If we want to speed up the dataset.map then we must specify the num_parallel_calls. 
  - Not defined : Process sequentially
  - tf.data.experimental.AUTOTUNE : parallel calls set based on available CPU 


In [46]:
def f(x):
  return x+2

print('Dataset.map(): All elements Increment by 1')
print(list(d_mem.map(lambda x: x+1, num_parallel_calls=2).as_numpy_iterator() ))
print(f'Using function ',list(d_mem.map(lambda x: f(x), num_parallel_calls=2).as_numpy_iterator() ))


print('\nDataset.reduce():  With initial intitial_state or starting_val')
print(f'=10 ', d_mem.reduce( 10,  lambda x,y: x+y).numpy() )
print(f'=0 ', d_mem.reduce( 0,  lambda x,y: x+y).numpy() )

Dataset.map(): All elements Increment by 1
[2, 3, 4]
Using function  [3, 4, 5]

Dataset.reduce():  With initial intitial_state or starting_val
=10  16
=0  6


## TF Transform : Using Python function
If you want to use the python function inside teh tensorflow pipeline, you can do so in two ways
  - Autograph: Use the function directly in the TF graphs. 
    - Pro: Easy to use
    - Con: Can covert some but not all python codes
  - tf.py_function : Define the python function as tf function during its use, indicating that it needs to converted into the tf code
    - Pro: Can write  arbitary code and will be supported by TF pipeline without throwing any error
    - Con: Generally results in worse performance
      - Not parallelised

In [47]:
def upper1(x: tf.Tensor):
  return x+10
  
person_modf = person.map( lambda x : tf.py_function( func=upper1, inp=[x['age']], Tout=tf.int32) )
print(list(person_modf.as_numpy_iterator()) )


[28, 40]


## TF.py_function vs TF.distribute.Server
The tf.py_function must run in the same address as the python program including the device, hence if you are using the distributed Tensorflow you must  use the tf.distribute.Server instead

## TF.Batch :
Create dataset batch to be used in the training
**[Caution]** : The batch sizes can be uneven because equal size batching may not be possible. e.g break 3 element into batch of 2. It may create warning/ errors due to different size. 

In [68]:
print(f'Uneven batch size :', list( d_mem.batch(2).as_numpy_iterator() ) )
print(f'\nEven batch size by dropping remainder :', \
      list( d_mem.batch(2, drop_remainder=True).as_numpy_iterator() ) )

print(f'\nEven batch size with padding :', \
      list( d_mem.batch(2).padded_batch(2, padded_shapes=[None]).as_numpy_iterator() ) )

Uneven batch size : [array([1, 2], dtype=int32), array([3], dtype=int32)]

Even batch size by dropping remainder : [array([1, 2], dtype=int32)]

Even batch size with padding : [array([[1, 2],
       [3, 0]], dtype=int32)]


## Optimisation
There are couple of things that can be done to speed up the.
  - prefetch :  If you are reading the data off the disk, then often the latency is involved in disk reads. You can avoid this latency and hence faster training by prefetching  some batch of data.

In [76]:
print('Orig d_mem :', list( d_mem.as_numpy_iterator() ))
# This one fetches all because we have only one  list in the dataset
print('Prefetches all ', list( d_mem.prefetch(1).as_numpy_iterator() ))
print('', list( d_mem.batch(1).prefetch(1).as_numpy_iterator() ))

Orig d_mem : [1, 2, 3]
Prefetches all  [1, 2, 3]
 [array([1], dtype=int32), array([2], dtype=int32), array([3], dtype=int32)]


In [0]:
# print(person.map(lambda d: (d['age'] , d['name'])))
# print(list(person.map(lambda d: (d['age'] , str(d['name'].numpy()).upper())).as_numpy_iterator() ))
# print(person.map( lambda x : upper(x) ).as_numpy_iterator()) 
# print(person.map( lambda x : upper(x) )) 
# print(list(person.as_numpy_iterator()))